# Agentic - Function Calling Using Bedrock and LangGraph


Install Dependencies

In [1]:
%pip install -qU boto3 langchain langchain-aws langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import (
    AnyMessage,
    SystemMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.tools import tool
from langchain_aws import ChatBedrock

## Define Tools
Now we define our tools. Tools are the actions that the agents can perform.

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun


@tool
def get_weather_condition(city: str):
    """Call to get the weather condition of a city.

    Args:
        city (str): The city name.

    Returns:
        str: The weather condition of the city.
    """
    search = DuckDuckGoSearchRun()
    query = f"weather condition of {city}"
    return search.run(query, max_results=1)


tools = [get_weather_condition]

## Define Agent and AgentState
Let's define our agent and agent state. The agent is the entity that performs actions, while the agent state tracks the agent's current condition. AgentState can utilize various persistent storage mechanisms; in this example, we use in-memory storage.

In [4]:
from langgraph.checkpoint.sqlite import SqliteSaver

# Using in-memory Sqlite database for saving the state.
memory = SqliteSaver.from_conn_string(":memory:")


# AgentState is a simple dictionary with a list of messages.
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.invoke_model)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def invoke_model(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling tool: {t} ...")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(
                    tool_call_id=t["id"], name=t["name"], content=str(result)
                )
            )
        print("Calling model with results...")
        return {"messages": results}

    def run(self, query, thread):
        messages = [HumanMessage(content=query)]

        for output in self.graph.stream({"messages": messages}, thread):
            for value in output.values():
                content = value["messages"][0].content
                if not content:
                    continue
                print(content)
                print("-" * 80)

In [6]:
prompt = """Your task is to assist based on the weather condition of a city.
Use the given tool to look up information. """

model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=dict(temperature=0),
)
agent = Agent(model, tools, system=prompt, checkpointer=memory)

Thread is a way to separate different conversations. Each Thread has its own context and state.

In [7]:
thread = {"configurable": {"thread_id": "1"}}

In [8]:
agent.run("Should I take umbrella today going to New York?", thread)

Calling tool: {'name': 'get_weather_condition', 'args': {'city': 'New York'}, 'id': 'toolu_bdrk_01G3ULpB3y5zH31fwxAXDseU', 'type': 'tool_call'} ...
Calling model with results...
Hazardous Weather Conditions. Hazardous Weather Outlook ; Flood Watch in effect from August 6, 02:00 PM EDT until August 7, 12:00 PM EDT; Current conditions at New York City, Central Park (KNYC) Lat: 40.78°NLon: 73.97°WElev: 154ft. Overcast. 79°F. 26°C. Humidity: 82%: Wind Speed: Calm: Barometer: 29.97 in (1013.9 mb) The latest forecast. Thunderstorms are expected to bring heavy rain and flood risks to New York City and the surrounding area Tuesday. CBS New York's First Alert Weather team has issued a Red ... Hazardous Weather Conditions. Hazardous Weather Outlook ; Flood Watch in effect from August 6, 02:00 PM EDT until August 7, 12:00 PM EDT; Current conditions at New York City, Central Park (KNYC) Lat: 40.78° N Lon: 73.97° W ... Hazardous Weather Conditions. Hazardous Weather Outlook ; Current conditions at 

We can test the agent's memory to make sure we are keeping track of the state.

In [9]:
agent.run("what was my first question?", thread)

Your first question was "Should I take umbrella today going to New York?"
--------------------------------------------------------------------------------


Now let's ask a question which our agent is not aware of, and has no defined tool to find it out.

In [10]:
agent.run("what is the Amazon stock price?", thread)

Unfortunately, I don't have a tool available to look up current stock prices. My capabilities are limited to the tools provided in this conversation, which is for checking weather conditions in a given city. I don't have a way to retrieve Amazon's stock price or other financial data without an appropriate tool or access to that information.
--------------------------------------------------------------------------------


## Multi-Tool Agent

Although it's a good practice to design agents in a loosely coupled and highly cohesive manner, sometimes it's necessary to have agents with multiple tools. We would like the model to be able to identify the relevant tool(s) based on the context and given query.

Let's add a few more tools to our agent. We'll add tools to perform some stock market related actions.

In [11]:
@tool
def get_current_stock_price(ticker: str) -> float:
    """
    Gets the current stock price for a given ticker.

    Parameters:
    ticker (str): The stock ticker symbol.

    Returns:
    float: The current stock price.
    """
    return 184.07


@tool
def get_stock_price_history(
    ticker: str, start_date: str, end_date: str
) -> list:
    """
    Gets the stock price history for a given ticker.

    Parameters:
    ticker (str): The stock ticker symbol.
    start_date (str): The start date of the price history.
    end_date (str): The end date of the price history.

    Returns:
    list: The stock price history. This is a list of floats.
    """
    return [181.71, 186.98, 184.07]


@tool
def buy_stock(ticker: str, quantity: int, price: float):
    """
    Buys a quantity of a stock at a given price.

    Parameters:
    ticker (str): The stock ticker symbol.
    quantity (int): The quantity of the stock to buy.
    price (float): The price at which to buy the stock.

    Returns:
    None
    """
    print(f"Bought {quantity} shares of {ticker} at ${price} each.")
    pass

We define a list of tools with stock related actions as well as our weather condition tool. 

In [12]:
tools = [
    get_current_stock_price,
    get_stock_price_history,
    buy_stock,
    get_weather_condition,
]

### Multiple Execution

In [13]:
prompt = """You are a financial assistant. Use the provided tools to get
information about stock prices and make trades.
You are allowed to make multiple calls (either together or in sequence).
"""
memory = SqliteSaver.from_conn_string(":memory:")

agent = Agent(model, tools, system=prompt, checkpointer=memory)

In [14]:
thread = {"configurable": {"thread_id": "1"}}

In [15]:
agent.run(
    query="What is the weather condition in New York? And what is stock price of Amazon between 2024-07-30 and 2024-08-01?",
    thread=thread,
)

Calling tool: {'name': 'get_weather_condition', 'args': {'city': 'New York'}, 'id': 'toolu_bdrk_0162vpqtmVXaVDkuV7GoskNQ', 'type': 'tool_call'} ...
Calling model with results...
Hazardous Weather Conditions. Hazardous Weather Outlook ; Flood Watch in effect from August 6, 02:00 PM EDT until August 7, 12:00 PM EDT; Current conditions at New York City, Central Park (KNYC) Lat: 40.78°NLon: 73.97°WElev: 154ft. Overcast. 79°F. 26°C. Humidity: 82%: Wind Speed: Calm: Barometer: 29.97 in (1013.9 mb) Hazardous Weather Conditions. Hazardous Weather Outlook ; Flood Watch in effect from August 6, 02:00 PM EDT until August 7, 12:00 PM EDT; Current conditions at New York City, Central Park (KNYC) Lat: 40.78° N Lon: 73.97° W ... Hazardous Weather Conditions. Hazardous Weather Outlook ; Current conditions at New York City, Central Park (KNYC) Lat: 40.78°NLon: 73.97°WElev: 154ft. Fair. 82°F. 28°C. Humidity: 56%: Wind Speed: ... New York NY . Today. High: 89 °F. Sunny then Slight Chance T-storms. Tonigh

In the example above, we asked the agent to perform multiple actions in a single query which were irrelevant to each other. The agent was able to identify the relevant tools and execute the actions accordingly.

Finally, the agent returned a result that combined the outputs of both actions.

### Nested Execution
In the following example, we ask the agent to perform a task that needs to be broken down into multiple dependent steps. The agent should then use the appropriate tool for each step and execute them in the correct order.

In [16]:
agent.run(
    query="Buy 20 shares of Amazon stock at the current price.", thread=thread
)

Calling tool: {'name': 'get_current_stock_price', 'args': {'ticker': 'AMZN'}, 'id': 'toolu_bdrk_016cviDgAXRwL6pSB9NV1niq', 'type': 'tool_call'} ...
Calling model with results...
184.07
--------------------------------------------------------------------------------
Calling tool: {'name': 'buy_stock', 'args': {'ticker': 'AMZN', 'quantity': 20, 'price': 184.07}, 'id': 'toolu_bdrk_01HYoYEubVFRA4QvkR9qyqHb', 'type': 'tool_call'} ...
Bought 20 shares of AMZN at $184.07 each.
Calling model with results...
None
--------------------------------------------------------------------------------
I have bought 20 shares of Amazon (AMZN) stock at the current price of $184.07 per share.
--------------------------------------------------------------------------------


In the above query, we asked agent to buy 20 shares at the current price.
1. Model realizes that it needs to first get the current price, so it calls `get_current_stock_price` tool.
2. Once it has the price, it calls `buy_stock` tool to buy the shares with the price returned in step 1.
3. Then finally it returns the combined output of all the steps.